In [1]:
import uproot
import awkward as ak #The events object is an awkward array
## Plotting.
import matplotlib.pyplot as plt

# Hists
import hist
from hist import Hist
import pickle #to save histograms

# NanoEvents
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema

# Processors (for parallelization)
#import coffea.processor as processor
from coffea import processor

#Math
import math

#Numpy
import numpy as np

#PackedSelection (to mask events)
from coffea.analysis_tools import PackedSelection

#Clock to count time
import time

In [2]:
#Function returning masks to select events of interest, used later in MyProcessor class
def ev_mask(old_events):
    #Implement the selections to veto events
    selection = PackedSelection()
        
    #Inclusive region for the HT, one charged lepton with pT > 15 GeV
    selection.add("inclusive_lepton_pT", ak.any(old_events.Electron.pt > 15.0, axis=1) | ak.any(old_events.Muon.pt > 15.0, axis=1))
        
    #At least one lepton 
    selection.add("one_l", (ak.num(old_events.Electron)+ak.num(old_events.Muon))>=1)
        
    #Electron signal region
    #The electron with higher pT (position 0) is tightly identified (Working Point 90) and isolated
    selection.add("fst_tight_e", ak.mask(old_events.Electron, ak.num(old_events.Electron) >= 1)[:, 0].mvaFall17V2Iso_WP90 == True)
    selection.add(
        "electron_pT",
        ak.mask(old_events.Electron, ak.num(old_events.Electron) >= 1)[:, 0].pt > 35.0
        #ak.any(old_events.Electron.pt > 35.0, axis=1)  
    )
    selection.add(
        "e_range_eta",
        ak.mask(old_events.Electron, ak.num(old_events.Electron) >= 1)[:, 0].eta < 2.5
        #ak.any(abs (old_events.Electron.eta) < 2.5, axis=1)
    )
    #The following cuts work in inverse logic, when you require the selection later you must take false
    #Veto events if there is a second tight electron
    selection.add("snd_tight_e", ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].mvaFall17V2Iso_WP90 == True)
    #Events containing a second loosely identified electron with pT > 10 GeV are vetoed.
    selection.add("loose_e", (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].mvaFall17V2Iso_WPL == True) & (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].pt >10))
    #Veto events if there is any tight muon
    selection.add("any_tight_mu", ak.any(old_events.Muon.tightId == True, axis=1))
    #Events containing a second loosely identified muon with pT > 10 GeV are vetoed
    selection.add("loose_mu1", (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 0].looseId == True) & (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 0].pt >10))
    selection.add("loose_mu2", (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].looseId == True) & (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].pt >10))
        
    #Muon signal region
    #The muon with higher pT (position 0) is tightly identified and isolated
    selection.add("fst_tight_mu", ak.mask(old_events.Muon, ak.num(old_events.Muon) >= 1)[:, 0].tightId == True)
    selection.add(
        "muon_pT",
        ak.mask(old_events.Muon, ak.num(old_events.Muon) >= 1)[:, 0].pt > 30.0
        #ak.any(old_events.Muon.pt >= 30.0, axis=1)
    )
    selection.add(
        "mu_range_eta",
        ak.mask(old_events.Muon, ak.num(old_events.Muon) >= 1)[:, 0].eta < 2.4
        #ak.any(abs (old_events.Muon.eta) < 2.4, axis=1)
    )
    #The following cuts work in inverse logic, when you require the selection later you must take false
    #Veto events if there is a second tight muon
    selection.add("snd_tight_mu", ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].tightId == True)
    #Events containing a second loosely identified muon with pT > 10 GeV are vetoed.
    selection.add("loose_mu", (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].looseId == True) & (ak.mask(old_events.Muon, ak.num(old_events.Muon) > 1)[:, 1].pt >10))
    #Veto events if there is any tight electron
    selection.add("any_tight_e", ak.any(old_events.Electron.mvaFall17V2Iso_WP90 == True, axis=1))
    #Events containing a second loosely identified electron with pT > 10 GeV are vetoed.
    selection.add("loose_e1", (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 0].mvaFall17V2Iso_WPL == True) & (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 0].pt >10))
    selection.add("loose_e2", (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].mvaFall17V2Iso_WPL == True) & (ak.mask(old_events.Electron, ak.num(old_events.Electron) > 1)[:, 1].pt >10))
    
    #Puppy (pileup-per-particle identification algorithm) met cut (requiring moderate amount of missing transverse momentum)
    selection.add("lead_pMET", old_events.PuppiMET.pt>30)
    #b Veto (no bjet candidates are found according to the loose working point of the DeepCSV tagger)
    selection.add("b_veto", ak.all(old_events.Jet.btagDeepB<=0.84, axis=1))
    #VBS jets (looking for the max invariant mass pair)
    # Get all combinations of jet pairs in every event
    dijets_o = ak.combinations(old_events.Jet, 2, fields=['i0', 'i1'])
    # Check that jet pairs have the greatest mass
    ismax_o=(dijets_o['i0']+dijets_o['i1']).mass==ak.max((dijets_o['i0']+dijets_o['i1']).mass, axis=1)
    #previous line: returns a Boolean array with True where the condition is met, and false otherwise
    # Mask the dijets with the ismax to get dijets with the gratest mass
    VBS_jets_o = dijets_o[ismax_o]
    # Separate pairs into arrays of the leading and the trailing VBS jets in each pair
    VBS_ljet_o, VBS_tjet_o = ak.unzip(VBS_jets_o)
    #VBS selection
    selection.add(
        "lead_VBS_ljet",   
        ak.all(VBS_ljet_o.pt>50, axis=1)
    )
    selection.add(
        "lead_VBS_tjet", 
        ak.all(VBS_tjet_o.pt>30, axis=1)
    )
    selection.add(
        "lead_VBS_mass",
        ak.all((VBS_ljet_o+VBS_tjet_o).mass>500, axis=1)
    )
    selection.add(
        "lead_VBS_eta",
        ak.all(abs(VBS_ljet_o.eta-VBS_tjet_o.eta)>2.5, axis=1)
    )
                      
    #An event is assigned to a boosted category if it contains only one AK8 jet (fatjet) as hadronically decaying vector boson V_had
    #together with at least two AK4 jets, originating from the scattered incoming partons,            
    selection.add("one_fatjet", (ak.num(old_events.FatJet) == 1))
    selection.add("two_jet", (ak.num(old_events.Jet) >= 2))
    #Invariant mass of the W boson decaying hadronically, use softdropmass (SD mass) 
    #(removes soft, wide-angle radiation from the large radius jet, improving the modeling of the jet mass observable)
    selection.add(
        "W_mass_fatjet",
        ak.all((old_events.FatJet.msoftdrop>70) & (old_events.FatJet.msoftdrop<115), axis=1)
    )
                      
    #Transverse mass of the W boson decaying leptonically
    #With electrons
    selection.add(
        "W_Tmass_e",
        ak.all(2*old_events.Electron.pt*old_events.MET.pt*(1-np.cos(old_events.Electron.phi-old_events.MET.phi))<185**2, axis=1)
    )
    #With muons
    selection.add(
        "W_Tmass_mu",
        ak.all(2*old_events.Muon.pt*old_events.MET.pt*(1-np.cos(old_events.Muon.phi-old_events.MET.phi))<185**2, axis=1)
    )
               
    #Inclusive mask (for HT)
    ev_mask_inclusive = selection.require(inclusive_lepton_pT=True)
    #Electron mask
    ev_mask_e = selection.require(one_l=True, fst_tight_e=True, snd_tight_e=False, loose_e=False, any_tight_mu=False, loose_mu1=False, loose_mu2=False, electron_pT=True, e_range_eta=True, lead_pMET=True, b_veto=True, lead_VBS_ljet=True, lead_VBS_tjet=True, lead_VBS_mass=True, lead_VBS_eta=True, one_fatjet=True, two_jet=True, W_Tmass_e=True, W_mass_fatjet=True)
    #Muon mask
    ev_mask_mu = selection.require(one_l=True, fst_tight_mu=True, snd_tight_mu=False, loose_mu=False, any_tight_e=False, loose_e1=False, loose_e2=False, muon_pT=True, mu_range_eta=True, lead_pMET=True, b_veto=True, lead_VBS_ljet=True, lead_VBS_tjet=True, lead_VBS_mass=True, lead_VBS_eta=True, one_fatjet=True, two_jet=True, W_Tmass_mu=True, W_mass_fatjet=True)
    #return mask
    return {"electron": ev_mask_e, "muon": ev_mask_mu, "inclusive": ev_mask_inclusive}

In [3]:
# The processor class bundles our data analysis together while giving us some helpful tools.  It also leaves looping and chunks to the framework instead of us.
class MyProcessor(processor.ProcessorABC):
    def __init__(self):
        pass
    
    def process(self, old_events):
        # This is where we do our actual analysis. The dataset has columns similar to the TTree's; events.columns can tell you them, or events.[object].columns for deeper depth.
        dataset = old_events.metadata['dataset']
        
        #Apply the cuts on the events with the mask, by using the function implemented earlier
        #(chose either electron or muon signal region)
        events=old_events[ev_mask(old_events)["electron"]]
        events_inclusive=old_events[ev_mask(old_events)["inclusive"]]
        #events_inclusive=old_events[ev_mask(old_events)["muon"]]
        
        #Defining the fiducial region
        jets=events.Jet
        fatjets = events.FatJet
        #Jets isolation from electrons
        # Get all combinations of jets and electrons in every event
        jets_e = ak.cartesian({"x": jets, "y": events.Electron})
        # Check that jets satisfy the isolation
        jets_iso_e = ((jets_e["x"].eta-jets_e["y"].eta)**2+(jets_e["x"].phi-jets_e["y"].phi)**2>0.4**2)
        # Mask the jets_e with the jets_iso to get jets isolated from electrons
        jets_e = jets_e[jets_iso_e]
        # Separate pairs into jets and electons, redefining the jets (but not the electrons)
        jets, el = ak.unzip(jets_e)
        #FatJets isolation from electrons
        # Get all combinations of fatjets and electrons in every event
        fatjets_e = ak.cartesian({"x": fatjets, "y": events.Electron})
        # Check that fatjets satisfy the isolation
        fatjets_iso_e = ((fatjets_e["x"].eta-fatjets_e["y"].eta)**2+(fatjets_e["x"].phi-fatjets_e["y"].phi)**2>0.8**2)
        # Mask the fatjets_e with the fatjets_iso to get fatjets isolated from electrons
        fatjets_e = fatjets_e[fatjets_iso_e]
        # Separate pairs into fatjets and electons, redefining the fatjets (but not the electrons)
        fatjets, el = ak.unzip(fatjets_e)
                      
        #Jets isolation from muons
        # Get all combinations of jets and muons in every event
        #jets_mu = ak.cartesian({"x": jets, "y": events.Muon})
        # Check that jets satisfy the isolation
        #jets_iso_mu = ((jets_mu["x"].eta-jets_mu["y"].eta)**2+(jets_mu["x"].phi-jets_mu["y"].phi)**2>0.4**2)
        # Mask the jets_mu with the jets_iso_mu to get jets isolated from muons
        #jets_mu = jets_mu[jets_iso_mu]
        # Separate pairs into jets and muons, redefining the jets (but not the muons)
        #jets, mu = ak.unzip(jets_mu)
        #FatJets isolation from muons
        # Get all combinations of fatjets and muons in every event
        #fatjets_mu = ak.cartesian({"x": fatjets, "y": events.Muon})
        # Check that fatjets satisfy the isolation
        #fatjets_iso_mu = ((fatjets_mu["x"].eta-fatjets_mu["y"].eta)**2+(fatjets_mu["x"].phi-fatjets_mu["y"].phi)**2>0.8**2)
        # Mask the fatjets_mu with the fatjets_iso_mu to get fatjets isolated from muons
        #fatjets_mu = fatjets_mu[fatjets_iso_mu]
        # Separate pairs into jets and muons, redefining the jets (but not the muons)
        #fatjets, mu = ak.unzip(fatjets_mu)
                      
        #Jets cuts
        jets_eta_cut = (abs (jets.eta) < 4.7)
        jets_pT_cut = (jets.pt > 30)
        jets = jets[jets_pT_cut&jets_eta_cut]
        #FatJets cuts
        fatjets_eta_cut = (abs (fatjets.eta) < 2.4)
        fatjets_pT_cut = (fatjets.pt > 200)
        fatjets = fatjets[fatjets_pT_cut&fatjets_eta_cut]
        #Removing AK4(Jet) jets overlapping with AK8(FatJets) jets
        # Get all combinations of jets and fatjets in every event
        jets_fatjets = ak.cartesian({"x": jets, "y": fatjets})
        # Check that jets satisfy the isolation
        jets_iso_f = ((jets_fatjets["x"].eta-jets_fatjets["y"].eta)**2+(jets_fatjets["x"].phi-jets_fatjets["y"].phi)**2>0.8**2)
        # Mask the jets_fatjets with the jets_iso_f to get jets isolated from fatjets
        jets_fatjets = jets_fatjets[jets_iso_f]
        # Separate pairs into jets and electons, redefining the jets (but not the fatjets)
        jets, fj = ak.unzip(jets_fatjets)
        #VBS jets (looking for the max invariant mass pair)
        # Get all combinations of jet pairs in every event
        VBSjets = ak.combinations(events.Jet, 2, fields=['i0', 'i1'])
        # Mask the dijets with the ismax to get dijets with the gratest mass
        VBS_jets = VBSjets[(VBSjets['i0']+VBSjets['i1']).mass==ak.max((VBSjets['i0']+VBSjets['i1']).mass, axis=1)]
        # Separate pairs into arrays of the leading and the trailing VBS jets in each pair.
        VBS_jet_lead, VBS_jet_trail = ak.unzip(VBS_jets)
        
        #Get the variables of N-subjectiness
        tau_1=fatjets.tau1
        tau_2=fatjets.tau2
        #Take the tau2 to tau1 ratio
        tau_r=tau_2/tau_1
        #Get the variables of particleNet and deepTag
        pNet=fatjets.particleNet_WvsQCD
        pNet_MD=(fatjets.particleNetMD_Xqq+fatjets.particleNetMD_Xcc)/(fatjets.particleNetMD_Xqq+fatjets.particleNetMD_Xcc+fatjets.particleNetMD_QCD)
        #for the Z boson, in the previous line you should add the Xbb
        dTag=fatjets.deepTag_WvsQCD
        
        # Bins and categories for the histogram
        h_PuppiMET = hist.Hist(hist.axis.Regular(20, 30, 300)) #(bins, start, stop)
        h_e_pT = hist.Hist(hist.axis.Regular(20, 35, 300))
        h_e_eta = hist.Hist(hist.axis.Regular(20, -2.5, 2.5))
        h_fatjet_pT = hist.Hist(hist.axis.Regular(20, 200, 600))
        h_fatjet_mass = hist.Hist(hist.axis.Regular(20, 30, 150))
        h_jets_mass = hist.Hist(hist.axis.Regular(24, 500, 3500))
        h_jets_eta = hist.Hist(hist.axis.Regular(20, 2.5, 8.5))
        h_jets_lead = hist.Hist(hist.axis.Regular(20, 50, 500))
        h_jets_trail = hist.Hist(hist.axis.Regular(14, 30, 250))
        h_LHE_HT = hist.Hist(hist.axis.Variable([70,100,200,400,600,800,1200,2500,3500]))
        h_Nsubj_t1 = hist.Hist(hist.axis.Regular(100, 0, 1))
        h_Nsubj_t2 = hist.Hist(hist.axis.Regular(100, 0, 1))
        h_t_ratio = hist.Hist(hist.axis.Regular(100, 0, 1))
        h_pNet = hist.Hist(hist.axis.Regular(100, 0, 1))
        h_pNet_MD = hist.Hist(hist.axis.Regular(100, 0, 1))
        h_dTag = hist.Hist(hist.axis.Regular(100, 0, 1))
          
        # This fills our histogram once our data is collected.
        h_PuppiMET.fill(events.PuppiMET.pt)
        h_e_pT.fill(events.Electron[:,0].pt)
        h_e_eta.fill(events.Electron[:,0].eta)
        h_fatjet_pT.fill(ak.flatten(fatjets.pt))
        #previous line: you have to flatten the awkward array because its size is ambiguous due to variable-length arrays
        h_fatjet_mass.fill(ak.flatten(fatjets.mass))
        h_jets_mass.fill(ak.flatten((VBS_jet_lead+VBS_jet_trail).mass))
        h_jets_eta.fill(ak.flatten(abs(VBS_jet_lead.eta-VBS_jet_trail.eta)))
        h_jets_lead.fill(ak.flatten(VBS_jet_lead.pt))
        h_jets_trail.fill(ak.flatten(VBS_jet_trail.pt))
        h_LHE_HT.fill(events_inclusive.LHE.HT)
        h_Nsubj_t1.fill(ak.flatten(tau_1))
        h_Nsubj_t2.fill(ak.flatten(tau_2))
        h_t_ratio.fill(ak.flatten(tau_r))
        h_pNet.fill(ak.flatten(pNet))
        h_pNet_MD.fill(ak.flatten(pNet_MD))
        h_dTag.fill(ak.flatten(dTag))
        
        #Ouput of the processor
        return {
            dataset: {
                "gen_events": len(old_events),
                "entries_inclusive": len(events_inclusive),
                "entries": len(events),
                "PuppiMET": h_PuppiMET,
                "e_pT": h_e_pT,
                "e_eta": h_e_eta,
                "fatjet_pT": h_fatjet_pT,
                "fatjet_mass": h_fatjet_mass,
                "jets_mass": h_jets_mass,
                "jets_eta": h_jets_eta,
                "jets_lead": h_jets_lead,
                "jets_trail": h_jets_trail,
                "LHE_HT": h_LHE_HT,
                "Nsubj_t1": h_Nsubj_t1,
                "Nsubj_t2": h_Nsubj_t2,
                "t_ratio": h_t_ratio,
                "pNet": h_pNet,
                "pNet_MD": h_pNet_MD,
                "dTag": h_dTag
            }
        }
    
    def postprocess(self, accumulator):
        #This is where we can make post-analysis adjustments, such as handle weights, rebinning or scaling our histograms.
        pass

In [4]:
#Clock
tstart = time.time()

#The processor to run the executor to use
futures_run = processor.Runner(
    executor = processor.FuturesExecutor(compression=None, workers=4),
    #In the previous line, we can then set the the number of cores to use as argument of the FuturesExecutor 
    schema=NanoAODSchema,
    #Coffea will split your data into chunks of many events.
    #chunksize=200_000 #default 100_000
    #maxchunks argument will stop the analysis after a certain number of chunks are reached
    #maxchunks=10,
)

out = futures_run(
    "fileset.json", #Fileset to read
    "Events", #Name of the TTree in the root file
    processor_instance=MyProcessor() #MyProcessor is the name of the class I defined above
)

elapsed = time.time() - tstart
print("Running time = {} s".format(elapsed))
#print(out)

Preprocessing:   0%|          | 0/647 [00:00<?, ?file/s]

Processing:   0%|          | 0/3282 [00:00<?, ?chunk/s]

Running time = 12597.81774687767 s


In [5]:
#Save output in file.pkl, need to import pickle
with open("output.pkl", "wb") as file_output:
    pickle.dump(out, file_output)
file_output.close()